In [15]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
import pytest
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter

#Check how to initialize imports para di na naka-indicate dito


In [16]:
#Set solver to gurobi
config = cobra.Configuration()
config.solver = 'gurobi'

#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6


In [17]:
#This codeblock is to define some functions that will be used for modelling.


#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='../misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
#     print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='../misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        row_m = str()
        row_bs = str()
        for rxns in rows:
            row_m += str(rxns) + "_M"
            row_bs += str(rxns) + "_BS"
        leaf_inactive_rxns.append(row_m)
        leaf_inactive_rxns.append(row_bs)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
#     print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")
def add_tissue_constraints(model):
    BS_photon_import = model.reactions.PRISM_white_LED_BS
    M_photon_import = model.reactions.PRISM_white_LED_M

    #Set photon flux ratio to 10:1
    photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)
    model.add_cons_vars(photon_flux)

    co2tex_m = model.reactions.CO2tex_M
    co2tex_bs = model.reactions.CO2tex_BS

    #CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
    #CO2 BS intake must be limited owing to its position in the leaf tissue

    # # #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
    # # Values from Von Caemmerer (2000) in intermediate C3-C4 photosynthetic plants is 1 mmol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
    # # #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
    # # #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
    co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:10, co2tex_bs.id:1},model)
    co2_ratio_cons.name = 'co2_ratio_cons'
    model.add_cons_vars(co2_ratio_cons)

    #o2 ratio constraint
    o2tex_bs = model.reactions.O2tex_BS
    o2tex_m = model.reactions.O2tex_M

    o2_ratio_cons = mm.set_fix_flux_ratio({o2tex_m.id:10, o2tex_bs.id:1},model)
    o2_ratio_cons.name = 'o2_ratio_cons'
    model.add_cons_vars(o2_ratio_cons)

    #no tissue-specific constraints will be set for Nitrates, h2o and other inorganic cofactors as 
    #they are transported via the apoplastic pathway rather than the symplast

    # #This code block contains constraints specific for enzyme rate constraints
    #This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
    #fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
    #At 0. For reversible reactions the lower bounds are set to the same value

def add_enzyme_constraints(model):
    #Maximum values for constraints
    wt_pepc = 0 #umol m-2 s-1 #Note: Need to constrain it to 0 pala since no PEPC was detected in either cell type
    wt_mdh = 11.18 #umol m-2 s-1
    wt_nadp_me = 0.14 #umol m-2 s-1
    wt_ppdk = 0.31 #umol m-2 s-1
    wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)

    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_BS = model.reactions.PPCc_BS
    pepc_M = model.reactions.PPCc_M

    wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                            + pepc_M.flux_expression, 
                                            lb = 0, ub = wt_pepc)
    wt_pepc_cons.name = 'wt_pepc_cons'
    model.add_cons_vars(wt_pepc_cons)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_BS = model.reactions.PPDKs_BS
    ppdks_M = model.reactions.PPDKs_M
    wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                             + ppdks_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    mdhm_M = model.reactions.MDHm_M


    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= 0, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    nadp_me_M = model.reactions.MDHys_M
    nadp_me_BS = model.reactions.MDHys_BS

    wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                             + nadp_me_BS.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional


    hco3es_m = model.reactions.HCO3Es_M.flux_expression
    hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
    hco3em_m = model.reactions.HCO3Em_M.flux_expression
    hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
    hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
    hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

    ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)
    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    rbpc_M = model.reactions.RBPCs_M.flux_expression
    rbpc_BS = model.reactions.RBPCs_BS.flux_expression
    rbpo_M = model.reactions.RBPOs_M.flux_expression
    rbpo_BS = model.reactions.RBPOs_BS.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                         - 1*(rbpc_M + rbpc_BS),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)

    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s_M.bounds = (0,0)
    model.reactions.RBPC2s_BS.bounds = (0,0)
    
    
    
    #What if I simply constrained that of the M cell one to 3:1?
    #This constraint is pretty good actually. 
    #This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
    #wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena (2021) (preprint) had done 

#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd): 
    ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
    ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
    ngam_atp_m.bounds = (0,1000)
    ngam_atp_bs.bounds = (0,1000)
    ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
    ngam_ratio.name = 'ngam_BS/M_ratio'
    model.add_cons_vars(ngam_ratio)

    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
    ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
    ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
    ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"

    nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
    nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
    nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
    nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)
    model.add_cons_vars(nadphox_c_s_BS)
    model.add_cons_vars(nadphox_s_m_BS)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

    fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
    fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
    fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
    fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                       + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                         - 1*(fex_atp_c_M + fex_atp_c_BS),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    #Compute NGAM value and add constraint as a lower bound/upper bound to model
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_ATP_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(sample_fluxes_df):
    print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
    print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
    print('RBPC2s_M', sample_fluxes['RBPC2s_M'], 'RBPC2s_BS', sample_fluxes['RBPC2s_BS'])
    print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
    print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
    print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
    print('co2 consumption M', sample_fluxes['CO2tex_M'], 'co2 consumption BS', sample_fluxes['CO2tex_BS'])
    print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
    print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
    print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs_M'], 'ATP synthase (mit) M', sample_fluxes['ATPSm_M'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)

This code block is to add the constraints for ngam nadphox, which for some reason doesn't save when it was generated using the curation script


In [18]:
# This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
#proper values? I think that would be a better approach, I think.


with model as m1:
    ppfd = 1500                                                                                                                                                                                                                                                                                         
    m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=0, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
    add_tissue_constraints(m1)
    add_enzyme_constraints(m1)
    add_ngam_cons(m1, ppfd)

    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    #Remove thermodynamically infeasible loops
    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
    sample_fluxes_df = sample_fluxes.to_frame()


print_summary(sample_fluxes_df)


rbcl M cell:  31.825903848218434 rbcl BS cell:  3.0222012294733838
rbcl M cell (photorespiration) 11.09517880635305 rbcl BS cell (PR) 0.5208562195442206
vc/vo M: 2.8684444301154537 vc/vo BS: 5.80237139554172
RBPC2s_M 0.0 RBPC2s_BS 0.0
PEPC M 0.0 PEPC BS 0.0
Carbonic Anhydrase (Cytosolic) M 0.0 Carbonic Anhydrase (Cytosolic) BS 0.0
NADP-ME M 0.0 NADP-ME BS 0.0
Biomass M:  0.0 Biomass BS 0.0
Phloem M:  0.0 Phloem BS 0.4670033170119195
co2 consumption M 26.363636363636708 co2 consumption BS 2.636363636363671
o2 consumption M 0.0 o2 consumption BS 0.0
Photosystem II M 78.80980536551006 PSII BS 6.985526913210046
PSI M 118.2125706287823 PSI BS 12.490411874244165
PPFD M:  837.46677979961 PPFD BS:  83.74667797996099
ATP synthesis (stromal) M 157.6167608383764 ATP synthase (mit) M 8.551665491362293
pd_abs_flux:  4.242858097778737


Part 2. Check Maximum NH4 consumption @ max ppfd to find upper bound
This part will use FVA to find the max and min exchange flux for NH3.



In [ ]:
%timeit

with model as m1:
    ppfd = 2000                                                                                                                                                                                                                                                                                         
    m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=0, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
    add_tissue_constraints(m1)
    add_enzyme_constraints(m1)
    add_ngam_cons(m1, ppfd)
    
   #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    #Perform FVA. Loopless mode is set to true while pfba factor is set to 1.1 to prevent futile cycling.
    FVA_soln = cobra.flux_analysis.flux_variability_analysis(m1, loopless=True, pfba_factor=1.1, processes=6)


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [ ]:

FVA_soln.loc['EX_nh4(e)']

#Based on FVA, the maximum amount of NH4 used by the system is 0.84 umol/m^2 s. 
#Therefore, this will be the maximum amount to be used.

In [21]:
#Test one instannce to see if it works
filepath = '../flux_results/N_benchmark/WT/'
filename = 'N-benchmark-growing-leaf.xls'
workbook = mm.create_xlsx(filename, filepath)


n_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))

with model as m1:

    m1.medium = define_model_medium(m1, co2=29, o2=2.216, ppfd=1500, h=0, nh4=0.84, no3=inf) #initialize model
    turn_off_cofac_cycles(m1)
    add_tissue_constraints(m1)
    add_enzyme_constraints(m1)
    add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula

    #Change objective function to mature leaf
    m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

    #Optimize then check optimality
    while True:
        solution = model.optimize()
        if solution.status == 'optimal':
            sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
            #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
            sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
            print("PPFD:", ppfd, "NH3 flux:", nitro, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
            #Append to Dataframe
            n_solns_2cell_fba[nh4] = sample_fluxes['fluxes']
            break
        else:
            continue
     
n_solns_2cell_fba.to_excel(workbook, sheet_name=str(1500))

workbook.close()

#It works naman, bakit di nagsesave?
#Ok it works na



Successfully generated N-benchmark-growing-leaf.xls-20221017-12:03 in ../flux_results/N_benchmark/WT/N-benchmark-growing-leaf.xls-20221017-12:03.xlsx
PPFD: 944.4444444444445 NH3 flux: 0.7466666666666666  RBPC_M cell flux: 31.85125749149504


The following code blocks are used to obtain a series of solutions under two (N limiting and Light-limiting) conditions, resulting with a 20x20xN matrix containing flux solutions for both Mature and Growing leaf models.

In [25]:
#For Mature Leaf (Phloem as Objective function)

#Define n and ppfd ranges for simulating n and light conditions respectively
n_range = np.linspace(0,0.84, 20)
ppfd_range = np.linspace(250, 1500, 20) #array([ 250.,  500.,  750., 1000., 1250., 1500.])

#First initialize excel spreadsheet
filepath = '../flux_results/N_benchmark/WT/'
filename = 'N-benchmark-Phloem-EX-leaf.xls'
workbook = mm.create_xlsx(filename, filepath)



for ppfd in ppfd_range:

    n_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))

    for nitro in n_range:

        #Generate instance of model every iteration
        with model as m1:

            m1.medium = define_model_medium(m1, co2=29, o2=2.216, ppfd=ppfd, h=0, nh4=nitro, no3=inf) #initialize model
            turn_off_cofac_cycles(m1)
            add_tissue_constraints(m1)
            add_enzyme_constraints(m1)
            add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula

            #Change objective function to mature leaf
            m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
            m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
            m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

            #Optimize then check optimality
            while True:
                solution = model.optimize()
                if solution.status == 'optimal':
                    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                    #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                    print("PPFD:", ppfd, "NH3 flux:", nitro, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                    #Append to Dataframe
                    n_solns_2cell_fba[nitro] = sample_fluxes['fluxes']
                    break
                else:
                    continue
                
    #Write matrix to excel workbook
        
    n_solns_2cell_fba.to_excel(workbook, sheet_name=str(ppfd))

workbook.close()

#Notes:
#Complete dataframe does not save
#I think I fixed it now, I saved each instance to the wrong column lol



Successfully generated N-benchmark-Phloem-EX-leaf.xls-20221017-12:19 in ../flux_results/N_benchmark/WT/N-benchmark-Phloem-EX-leaf.xls-20221017-12:19.xlsx
PPFD: 250.0 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 250.0 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 1.6733322841371074
PPFD: 250.0 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 3.3657846321430522
PPFD: 250.0 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 5.046184791930571
PPFD: 250.0 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 6.674424313603228
PPFD: 250.0 NH3 flux: 0.22105263157894736  RBPC_M cell flux: 8.376861474625546
PPFD: 250.0 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 8.698856698310921
PPFD: 250.0 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 8.698856698319354
PPFD: 250.0 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 8.698856698311053
PPFD: 250.0 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 8.698856698311124
PPFD: 250.0 NH3 flux: 0.4421052631578947  RBPC_M cell flux: 8.698856698310692
PPFD: 250

PPFD: 513.1578947368421 NH3 flux: 0.6189473684210526  RBPC_M cell flux: 18.156710209426972
PPFD: 513.1578947368421 NH3 flux: 0.6631578947368421  RBPC_M cell flux: 18.15671020942629
PPFD: 513.1578947368421 NH3 flux: 0.7073684210526315  RBPC_M cell flux: 18.15671020942673
PPFD: 513.1578947368421 NH3 flux: 0.751578947368421  RBPC_M cell flux: 18.156710209426564
PPFD: 513.1578947368421 NH3 flux: 0.7957894736842105  RBPC_M cell flux: 18.156710209426205
PPFD: 513.1578947368421 NH3 flux: 0.84  RBPC_M cell flux: 18.156710209426954
PPFD: 578.9473684210526 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 578.9473684210526 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 1.6816006122079141
PPFD: 578.9473684210526 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 3.3533489612930585
PPFD: 578.9473684210526 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 5.031999157567608
PPFD: 578.9473684210526 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 6.7246607184518
PPFD: 578.9473684210526 NH3 flux: 0.22105263157

PPFD: 842.1052631578947 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 11.69802144273889
PPFD: 842.1052631578947 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 13.401414535248126
PPFD: 842.1052631578947 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 15.083881647707695
PPFD: 842.1052631578947 NH3 flux: 0.4421052631578947  RBPC_M cell flux: 16.766348760163357
PPFD: 842.1052631578947 NH3 flux: 0.4863157894736842  RBPC_M cell flux: 18.449476329721925
PPFD: 842.1052631578947 NH3 flux: 0.5305263157894736  RBPC_M cell flux: 20.161488130446187
PPFD: 842.1052631578947 NH3 flux: 0.5747368421052631  RBPC_M cell flux: 21.835823887732012
PPFD: 842.1052631578947 NH3 flux: 0.6189473684210526  RBPC_M cell flux: 23.51995127032947
PPFD: 842.1052631578947 NH3 flux: 0.6631578947368421  RBPC_M cell flux: 25.209689150137695
PPFD: 842.1052631578947 NH3 flux: 0.7073684210526315  RBPC_M cell flux: 26.899974095807167
PPFD: 842.1052631578947 NH3 flux: 0.751578947368421  RBPC_M cell flux: 28.589930045475626


PPFD: 1105.2631578947367 NH3 flux: 0.84  RBPC_M cell flux: 31.840772077962296
PPFD: 1171.0526315789473 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 1171.0526315789473 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 1.681600612207873
PPFD: 1171.0526315789473 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 3.34523583833802
PPFD: 1171.0526315789473 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 5.035793480148418
PPFD: 1171.0526315789473 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 6.725566828529263
PPFD: 1171.0526315789473 NH3 flux: 0.22105263157894736  RBPC_M cell flux: 8.382584444724461
PPFD: 1171.0526315789473 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 10.085819880794643
PPFD: 1171.0526315789473 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 11.778508198849254
PPFD: 1171.0526315789473 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 13.333339476368645
PPFD: 1171.0526315789473 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 15.07006034682104
PPFD: 1171.0526315789473 NH3 flux

PPFD: 1434.2105263157894 NH3 flux: 0.4863157894736842  RBPC_M cell flux: 18.424032960018327
PPFD: 1434.2105263157894 NH3 flux: 0.5305263157894736  RBPC_M cell flux: 20.106800071320208
PPFD: 1434.2105263157894 NH3 flux: 0.5747368421052631  RBPC_M cell flux: 21.789267183774278
PPFD: 1434.2105263157894 NH3 flux: 0.6189473684210526  RBPC_M cell flux: 23.471734296232228
PPFD: 1434.2105263157894 NH3 flux: 0.6631578947368421  RBPC_M cell flux: 25.15420140868853
PPFD: 1434.2105263157894 NH3 flux: 0.7073684210526315  RBPC_M cell flux: 26.83852501571922
PPFD: 1434.2105263157894 NH3 flux: 0.751578947368421  RBPC_M cell flux: 28.562896020519663
PPFD: 1434.2105263157894 NH3 flux: 0.7957894736842105  RBPC_M cell flux: 30.260040638283424
PPFD: 1434.2105263157894 NH3 flux: 0.84  RBPC_M cell flux: 31.811027018920342
PPFD: 1500.0 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 1500.0 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 1.6836411332831154
PPFD: 1500.0 NH3 flux: 0.08842105263157894  RBPC_M cell fl

In [26]:
#For Growing leaf

#Define n and ppfd ranges for simulating n and light conditions respectively
n_range = np.linspace(0,0.84, 20)
ppfd_range = np.linspace(250, 1500, 20) #array([ 250.,  500.,  750., 1000., 1250., 1500.])

#First initialize excel spreadsheet
filepath = '../flux_results/N_benchmark/WT/'
filename = 'N-benchmark-Biomass-EX-leaf.xls'
workbook = mm.create_xlsx(filename, filepath)



for ppfd in ppfd_range:

    n_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))

    for nitro in n_range:

        #Generate instance of model every iteration
        with model as m1:

            m1.medium = define_model_medium(m1, co2=29, o2=2.216, ppfd=ppfd, h=0, nh4=nitro, no3=inf) #initialize model
            turn_off_cofac_cycles(m1)
            add_tissue_constraints(m1)
            add_enzyme_constraints(m1)
            add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula

            #Change objective function to mature leaf
            m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 1
            m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 1
            m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 0

            #Optimize then check optimality
            while True:
                solution = model.optimize()
                if solution.status == 'optimal':
                    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                    #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                    print("PPFD:", ppfd, "NH3 flux:", nitro, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                    #Append to Dataframe
                    n_solns_2cell_fba[nitro] = sample_fluxes['fluxes']
                    break
                else:
                    continue
                
    #Write matrix to excel workbook
        
    n_solns_2cell_fba.to_excel(workbook, sheet_name=str(ppfd))

workbook.close()

#Notes:
#Complete dataframe does not save
#I think I fixed it now, I saved each instance to the wrong column lol



Successfully generated N-benchmark-Biomass-EX-leaf.xls-20221017-12:52 in ../flux_results/N_benchmark/WT/N-benchmark-Biomass-EX-leaf.xls-20221017-12:52.xlsx
PPFD: 250.0 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 250.0 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 4.238072167820478
PPFD: 250.0 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 7.319904168392961
PPFD: 250.0 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 7.319904168392972
PPFD: 250.0 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 7.319904168393615
PPFD: 250.0 NH3 flux: 0.22105263157894736  RBPC_M cell flux: 7.319904168393372
PPFD: 250.0 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 7.319904168393362
PPFD: 250.0 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 7.319904168393362
PPFD: 250.0 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 7.319904168393849
PPFD: 250.0 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 7.319904168393849
PPFD: 250.0 NH3 flux: 0.4421052631578947  RBPC_M cell flux: 7.319904168393849
PPFD: 250

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


PPFD: 513.1578947368421 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 12.984279741403887
PPFD: 513.1578947368421 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 8.403697191491888
PPFD: 513.1578947368421 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 12.672031009582595
PPFD: 513.1578947368421 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 15.585255628256942
PPFD: 513.1578947368421 NH3 flux: 0.22105263157894736  RBPC_M cell flux: 15.586743293900419
PPFD: 513.1578947368421 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 15.586743293901161
PPFD: 513.1578947368421 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 15.586743293895243
PPFD: 513.1578947368421 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 15.586743293890823
PPFD: 513.1578947368421 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 15.586743293901467
PPFD: 513.1578947368421 NH3 flux: 0.4421052631578947  RBPC_M cell flux: 15.586743293901467
PPFD: 513.1578947368421 NH3 flux: 0.4863157894736842  RBPC_M cell flux: 15.5867432939

PPFD: 776.3157894736842 NH3 flux: 0.5747368421052631  RBPC_M cell flux: 23.75419588179392
PPFD: 776.3157894736842 NH3 flux: 0.6189473684210526  RBPC_M cell flux: 23.754195880685742
PPFD: 776.3157894736842 NH3 flux: 0.6631578947368421  RBPC_M cell flux: 23.75419588175763
PPFD: 776.3157894736842 NH3 flux: 0.7073684210526315  RBPC_M cell flux: 23.754195881639347
PPFD: 776.3157894736842 NH3 flux: 0.751578947368421  RBPC_M cell flux: 23.75419588177639
PPFD: 776.3157894736842 NH3 flux: 0.7957894736842105  RBPC_M cell flux: 23.75419588177639
PPFD: 776.3157894736842 NH3 flux: 0.84  RBPC_M cell flux: 23.75419588177639
PPFD: 842.1052631578947 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 842.1052631578947 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 4.254695206573931
PPFD: 842.1052631578947 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 8.469194920774193
PPFD: 842.1052631578947 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 12.682133389643212
PPFD: 842.1052631578947 NH3 flux: 0.1768421052631

PPFD: 1105.2631578947367 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 25.336380618770477
PPFD: 1105.2631578947367 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 29.56229676737035
PPFD: 1105.2631578947367 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 32.39125736576266
PPFD: 1105.2631578947367 NH3 flux: 0.39789473684210525  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.4421052631578947  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.4863157894736842  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.5305263157894736  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.5747368421052631  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.6189473684210526  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.6631578947368421  RBPC_M cell flux: 32.52346460100546
PPFD: 1105.2631578947367 NH3 flux: 0.7073684210526315  RBPC_M cell flux: 32.52346460100

PPFD: 1368.4210526315787 NH3 flux: 0.7957894736842105  RBPC_M cell flux: 32.38750742983239
PPFD: 1368.4210526315787 NH3 flux: 0.84  RBPC_M cell flux: 32.38750742983239
PPFD: 1434.2105263157894 NH3 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 1434.2105263157894 NH3 flux: 0.04421052631578947  RBPC_M cell flux: 4.246455207230332
PPFD: 1434.2105263157894 NH3 flux: 0.08842105263157894  RBPC_M cell flux: 8.435078005134105
PPFD: 1434.2105263157894 NH3 flux: 0.1326315789473684  RBPC_M cell flux: 12.700317673736096
PPFD: 1434.2105263157894 NH3 flux: 0.17684210526315788  RBPC_M cell flux: 16.913256142598424
PPFD: 1434.2105263157894 NH3 flux: 0.22105263157894736  RBPC_M cell flux: 21.126194611472865
PPFD: 1434.2105263157894 NH3 flux: 0.2652631578947368  RBPC_M cell flux: 25.227608290672777
PPFD: 1434.2105263157894 NH3 flux: 0.3094736842105263  RBPC_M cell flux: 29.558539563543455
PPFD: 1434.2105263157894 NH3 flux: 0.35368421052631577  RBPC_M cell flux: 32.519837645669305
PPFD: 1434.2105263157894 NH3 fl